In [ ]:
import pandas as pd
import numpy as np
import json
from jinja2 import Environment, FileSystemLoader

In [ ]:
def table_data_format(df_orig):
    metadata = []
    df = df_orig
    df.replace(to_replace=["na", "?", np.nan, "missing", "not available",
                           "n/a", "missing value"], value=np.nan, inplace=True)

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')

    for idx, col in enumerate(df.columns):
        helper = {}
        key = col
        if len(key) > 15:
            key = col[:15]

        helper['ascend'] = 0
        helper['name'] = key
        if(df.dtypes[col] == "object"):
            df[col].replace(to_replace=["na", "?", np.nan, "missing", "not available",
                                        "n/a", "missing value"], value="missing value", inplace=True)
            helper['datatype'] = 'string'
            helper['values'] = df[col].unique().tolist()
        elif(df.dtypes[col] == "int64"):
            helper['datatype'] = 'int'
            helper['min_val'] = df[col].min()
            helper['max_val'] = df[col].max()
        elif(df.dtypes[col] == "float64"):
            helper['datatype'] = 'float'
            helper['min_val'] = df[col].min()
            helper['max_val'] = df[col].max()
        metadata.append(helper)

    df.replace(to_replace=["na", "?", np.nan, "missing", "not available",
                           "n/a", "missing value"], value="missing value", inplace=True)
    data_array = []
    for row in range(df.shape[0]):
        aux = []
        for col in range(df.shape[1]):
            aux.append(df.iloc[row, col])
        data_array.append(aux)
    return metadata, data_array

In [ ]:
path = r'C:\Users\Luis_Dutra\3D Objects\datasets\enem\enem_2019_for_table_viz.csv'

# Ler quantidade determinada de linhas do dataset
df = pd.read_csv(path,  encoding='iso8859-1')

df

In [ ]:
del df['NU_INSCRICAO']

In [ ]:
# Adiciona o  metadata_table_viz e dataarray_table_viz no html

file_loader = FileSystemLoader(r'C:\Users\Luis_Dutra\Documents\REPOS\pandas-profiling\pandas_profiling\templates')
env = Environment(loader=file_loader)

template = env.get_template("table.html")

keys, data_array = table_data_format(df)

output = template.render(metadata_table_viz=keys, dataarray_table_viz=data_array)

# C:\pandas-profiling\notebook
with open("test_500.html", "w") as result_file:
    result_file.write(output)